## Setting up Keys

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

In [1]:
# imports
import os
import openai
import requests
from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import Markdown, display, update_display

c:\Users\tazeb\OneDrive\AtomicHabit\LLM Engineering\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=google_api_key)

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyCx


## Asking LLMs to tell a Information/story/Jokes ---etc

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [3]:
system_message = "You are an assistant that is great at telling storeis/informations and or ideas"
user_prompt = "tell me a vary short information about Lalibella at Ethiopian Orthodox church and generate/find the link of its building in Addis Ababa "

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

#### A) GPT model Response

In [4]:
# 1.  GPT-3.5-Turbo
completion = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Lalibela is a town in Ethiopia known for its rock-hewn churches, particularly the UNESCO World Heritage site of Lalibela Ethiopian Orthodox Church. These churches were carved entirely out of solid rock in the 12th century. The Lalibela Ethiopian Orthodox Church is a significant pilgrimage site for Christians.

To find more information about Lalibela Ethiopian Orthodox Church, you can visit the following link: [Lalibela Ethiopian Orthodox Church in Addis Ababa](https://www.google.com/maps/search/Lalibela+Ethiopian+Orthodox+Church+Addis+Ababa)


In [5]:
# 2. GPT-4o-mini
# Temperature setting controls creativity
completion = openai.ChatCompletion.create(model='gpt-4o-mini', 
                                          messages=prompts, 
                                          temperature=0.7)
print(completion.choices[0].message.content)

Lalibela is a town in Ethiopia renowned for its rock-hewn churches, which were carved out of solid rock in the 12th century. It is often referred to as "the New Jerusalem" due to its significance in Ethiopian Orthodox Christianity. The churches are a UNESCO World Heritage Site and are considered a pilgrimage site for many Christians.

As for the link to its building in Addis Ababa, it's important to clarify that Lalibela is a distinct town located about 645 kilometers north of Addis Ababa, rather than a building within the capital. However, if you're looking for information about Ethiopian Orthodox churches in Addis Ababa, one notable site is the Holy Trinity Cathedral, which serves as the national church of Ethiopia.

You can find more information about Lalibela and its churches here: [UNESCO World Heritage - Lalibela](https://whc.unesco.org/en/list/18)


In [6]:
# 3. GPT-4o
completion = openai.ChatCompletion.create(model='gpt-4o', 
                                          messages=prompts, 
                                          temperature=0.4)
print(completion.choices[0].message.content)

Lalibela is a town in northern Ethiopia renowned for its rock-hewn churches, which are considered one of the greatest architectural achievements of the Ethiopian Orthodox Church. These churches, carved directly into the rock in the 12th century, are a significant pilgrimage site for Ethiopian Christians and are recognized as a UNESCO World Heritage Site. The most famous of these is the Church of St. George, which is shaped like a cross and carved from a single rock.

Regarding the Ethiopian Orthodox Church in Addis Ababa, one of the most prominent churches is the Holy Trinity Cathedral (Kidist Selassie), which serves as the highest-ranking Ethiopian Orthodox Tewahedo cathedral in Addis Ababa. Unfortunately, I can't generate or provide direct links, but you can search for "Holy Trinity Cathedral Addis Ababa" on a map service or search engine for more details and directions.


In [7]:
# Convert OpenAI-style prompts to Gemini-compatible input
gemini_prompt = "\n".join([msg["content"] for msg in prompts])
# Initialize the Gemini model
model = genai.GenerativeModel("gemini-pro")
# Generate a response
response = model.generate_content(gemini_prompt)
# Print the output
if response.candidates and response.candidates[0].content.parts:
    print(response.candidates[0].content.parts[0].text)
else:
    print("Response blocked due to content moderation.")

**About Lalibela and the Ethiopian Orthodox Church:**

Lalibela is a town in northern Ethiopia renowned for its 12th-century rock-hewn churches. These UNESCO World Heritage sites are carved directly into monolithic blocks of red volcanic tuff and are considered among the most remarkable examples of medieval architecture. The churches are named after King Lalibela, who is said to have built them in the 12th century after a pilgrimage to Jerusalem. The churches are still used by the Ethiopian Orthodox Church for religious services and pilgrimages.

**Link to the Building of the Ethiopian Orthodox Church in Addis Ababa:**

* [Holy Trinity Cathedral](https://www.google.com/maps/place/Holy+Trinity+Cathedral/@9.0221975,38.7432376,15z/data=!4m5!3m4!1s0x1659d4c9440250d7:0x7d25a4b26e9950e!8m2!3d9.0221975!4d38.7432376)


In [8]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

# Define API URL
api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key={google_api_key}"

# Prepare payload (Gemini API expects a simple prompt format)
payload = {
    "contents": [{"parts": [{"text": f"{system_message}\n\n{user_prompt}"}]}]
}

# Call Gemini API
response = requests.post(api_url, json=payload)
result = response.json()

# Extract and print response
if "candidates" in result and result["candidates"]:
    print(result["candidates"][0]["content"]["parts"][0]["text"])
else:
    print("Error: No valid response received.")

**Lalibela at Ethiopian Orthodox Church**

Lalibela is a town in northern Ethiopia famous for its monolithic churches carved out of a single rock. These churches were built in the 12th and 13th centuries by King Lalibela and are considered one of the most important pilgrimage sites for Ethiopian Orthodox Christians. The churches are adorned with intricate carvings and paintings and are a UNESCO World Heritage Site.

**Link to the building in Addis Ababa:**

https://www.google.com/maps/place/Lalibela+Church,+Addis+Abeba,+Ethiopia/@9.027126,38.7509052,17z/data=!3m1!4b1!4m5!3m4!1s0x164b76f9945f8db1:0x98c24391f3dba3d0!8m2!3d9.027126!4d38.7530939
